In [17]:
ls

ca.py             run_bert_ft_classifier.py
CHANGELOG.md      run_bert_training_classifier.py
cnn_kd.py         run_cnn_classifier.py
config.py         run_kd_lstm_atten_classifier.py
core.24141        run_kd_lstm_classifier.py
dataloader.py     run_lstm_baseline_classifier.py
data_prepare.py   run_lstm_classifier.py
earlystopping.py  run_lstm_kd_classifier.py
eval.py           testing_kd.py
model.py          test.py
models.py         train.py
__pycache__/      Untitled.ipynb
README.md         utils.py
run_atten.py


In [18]:
cd ..

/project/6006178/dongxx


In [19]:
ls

16_mercer_kd.out   kd_lstm.out         slurm-22060427.out  t_cnn.sh
16_pa_kd.out       LSTMbaseline/       slurm-22060432.out  Tiwtter_atten_kd.out
32_mercer_kd.out   Model_parameter/    slurm-22205575.out  Tiwtter_atten.out
32_pa_kd.out       new_bert.pt         slurm-22205595.out  Tiwtter_cnn_kd.out
atten.out          r_bert_gru.sh       slurm-22290702.out  Tiwtter_cnn.out
bert_ft.sh         r_bert_t.sh         slurm-22290962.out  Tiwtter_gru_bert.out
data/              result/             slurm-22291353.out  Tiwtter_linear.out
data140/           r_lstm_a.sh         slurm-22407206.out  Tiwtter_lstm_kd.out
ENV/               r_lstm.sh           slurm-28010453.out  Tiwtter_lstm.out
ft_kd_atten.out    slurm-21749546.out  slurm-28010456.out  t_lstm_kd.sh
glove.6B.100d.txt  slurm-21990450.out  t_atten_lstm_kd.sh  t_lstm.sh
IMDB_data/         slurm-21991185.out  t_att_lstm.sh       twitter_baseline/
IMDB_result/       slurm-21992242.out  t_bert_t.sh         twitter_parameter/
kd_atten_lstm

In [20]:
cd LSTMbaseline/

/project/6006178/dongxx/LSTMbaseline


In [21]:
from models import CNN_Baseline,LSTMBaseline
from run_lstm_baseline_classifier import validate, prepare_dateset,generate_batch,weight_matrix,epoch_time
import torch
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchtext.vocab import GloVe,Vocab,Vectors
from tqdm import tqdm
from utils import IMDB_indexing, pad_sequenc,pad_sequencing
from models import CNN_Baseline,LSTMBaseline
import torchtext.vocab
import csv
import pandas as pd
import argparse
import logging
import os
import pickle
import sys
import config
config.seed_torch()
from collections import Counter
import time
import copy

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [23]:
test = '/home/dongxx/projects/def-parimala/dongxx/data/test.csv'
valid = '/home/dongxx/projects/def-parimala/dongxx/data/valid.csv'
train ='/home/dongxx/projects/def-parimala/dongxx/data/train.csv'

In [24]:
counter2 = Counter({'<unk>': 400002, '<pad>': 400001})
glove = Vectors(name='../glove.6B.100d.txt')
f = open('../glove.6B.{}d.txt'.format(100), 'r')
loop = tqdm(f)
vob = {}
loop.set_description('Load Glove')
for i,line in enumerate(loop):
    values = line.split()
    word = values[0]
    vob[word] =400000-i
counter1 = copy.deepcopy(vob)
f.close()
counter1.update(counter2)
vocab = Vocab(counter1)

Load Glove: : 400000it [00:07, 56156.04it/s]


In [25]:
train_dataset, validation_dataset,test_dataset,labellist = prepare_dateset(train, valid,test,vocab)
testing = DataLoader(test_dataset, collate_fn= generate_batch, batch_size=32, shuffle=False)
LSTM_model =LSTMBaseline(vocab_size = 400002,hidden_dim =256 , n_layers = 2, dropout = 0.25, number_class = 2, bidirectional = True, embedding_dim =100)
LSTM_model.embedding_layer.weight.data.copy_(weight_matrix(vocab,glove)).to(device)
LSTM_model.embedding_layer.weight.data[1] = torch.zeros(100)
LSTM_model.embedding_layer.weight.data[0] = torch.zeros(100)
LSTM_model.to(device)
criterion = nn.CrossEntropyLoss()
criterion.to(device)
LSTM_model.load_state_dict(torch.load('/home/dongxx/projects/def-parimala/dongxx/Model_parameter/kd_lstm.pt'))

Start loading training data
Finish loading training data
Start loading validation data
Finish loading validation data
Start loading testing data
Finish loading testing data
prepare training and test sets


20000lines [00:05, 3853.69lines/s]
5000lines [00:01, 4122.06lines/s]
25000lines [00:06, 4112.25lines/s]


building vocab


<All keys matched successfully>

In [ ]:
start_time = time.time()
test_loss, test_acc,flat_list = validate(testing,LSTM_model,criterion,device)
end_time = time.time()
epoch_mins, epoch_secs = epoch_time(start_time, end_time)
print(f' | Epoch Time: {epoch_mins}m {epoch_secs}s')



In [ ]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

In [ ]:
pred_list =[item.cpu().numpy() for sublist in flat_list for item in sublist]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
cnf_matrix = confusion_matrix(labellist, pred_list)
plt.figure(figsize=(12,12))
plot_confusion_matrix(cnf_matrix, classes=[0,1], title="Confusion matrix")
plt.show()

In [ ]:
print(classification_report(labellist, pred_list))